#### Data Generation with your own reactions, modify
#### dataframe.to_csv('datasets/{rock/paper/scissor}.csv', header=True, index=None)
#### to save the properly reactions in the folders to train the model afterward
#### also modify "new_dataframe['target'] = 2" por 0=rock, 1=paper, 2=scissor, 
#### to define the dependent variable that the model must predict 

In [9]:
import cv2
import mediapipe as mp
import csv

import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

dataframe = None

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        landmarks_hand = {}

        if results.multi_hand_landmarks:

            for index, landmark in enumerate(results.multi_hand_landmarks[0].landmark):
                x = landmark.x
                y = landmark.y
                z = landmark.z

                landmarks_hand[f'landmark_{index}_x'] = x
                landmarks_hand[f'landmark_{index}_y'] = y
                landmarks_hand[f'landmark_{index}_z'] = z

            new_dataframe = pd.DataFrame(landmarks_hand, index=range(len(landmarks_hand.values())))
            new_dataframe['target'] = 2

            if dataframe is not None:
                dataframe = pd.concat((dataframe, new_dataframe))
            else:
                dataframe = new_dataframe.copy()

            dataframe.to_csv('datasets/scissor.csv', header=True, index=None)

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

rock = pd.read_csv('datasets/rock.csv')
scissor = pd.read_csv('datasets/scissor.csv')
paper = pd.read_csv('datasets/paper.csv')

original_dataframe = pd.concat((rock, paper, scissor))
original_dataframe = original_dataframe.sample(frac=1)

x = original_dataframe.drop('target', axis=1)
y = original_dataframe['target']

one_hot = OneHotEncoder(sparse_output=False)
y = one_hot.fit_transform(y.to_numpy().reshape(-1, 1))

print(y)
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.7)

test_x, ev_x, test_y, ev_y = train_test_split(x, y, train_size=0.7)

train_x

In [11]:
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout, Convolution2D
from keras import Sequential
import tensorflow as tf

model = Sequential()
model.add(Dense(63, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [12]:
print(x.shape)

(18333, 63)


In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['Accuracy'])

In [ ]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10)

In [ ]:
predict = ev_x.iloc[0].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[0])

predict = ev_x.iloc[10].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[10])

predict = ev_x.iloc[13].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[13])

In [ ]:
import numpy as np

def get_machine_move():
    return np.random.choice([0,1,2])

def check_move(imagen, machine_move, prediction):
    if machine_move == 0:
        if prediction == 0:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(imagen, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        elif prediction == 1:
            text = 'has ganado yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        else:
            text = 'has perdido yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

    elif machine_move == 1:
        if prediction == 0:
            text = 'has perdido yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        elif prediction == 1:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text
        else:
            text = 'has ganado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

    else:
        if prediction == 0:
            text = 'has ganado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

        elif prediction == 1:
            text = 'has perdido yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text
        else:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
type_predict = {0: 'rock', 1:'paper', 2:'scissors'}
image_predict = {0: cv2.imread("filtros/piedra.jpg"), 1:cv2.imread("filtros/paper.jpg"), 2:cv2.imread("filtros/tijeras.jpg")}
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 0)
thickness = 1
wait = False
dataframe = None
count = 0
cap = cv2.VideoCapture(0)
timer = 3
wait_5_seconds = False
win_text = ''
win_image = np.array([])

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        count += 1
        success, image = cap.read()

        image = cv2.flip(image, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.putText(image, str(timer), (int(image.shape[1]/2), int(image.shape[0]/2)), font, fontScale, color, thickness, cv2.LINE_AA)

        if wait_5_seconds and count < 200:
            cv2.putText(image, win_text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

        elif count >= 200 and wait_5_seconds:
            wait_5_seconds = False
            count = 0
            timer = 3
            win_text = ''
            win_image = np.array([])

        if count >= 30 and timer > 0:
            timer-=1
            count = 0

        if timer == 0 and not wait_5_seconds:
            results = hands.process(image)

            landmarks_hand = {}

            if results.multi_hand_landmarks:

                for index, landmark in enumerate(results.multi_hand_landmarks[0].landmark):
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z

                    landmarks_hand[f'landmark_{index}_x'] = x
                    landmarks_hand[f'landmark_{index}_y'] = y
                    landmarks_hand[f'landmark_{index}_z'] = z

                machine_move = get_machine_move()

                predict = np.array(list(landmarks_hand.values())).reshape(1, -1)
                predict = np.argmax(model.predict(predict))
                print(predict)
                win_text = check_move(image, machine_move, predict)

                win_image = image_predict[machine_move]
                win_image = cv2.resize(win_image, (200, 200))
                cv2.imshow('Machine', win_image)

                count = 0
                wait_5_seconds = True

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()